In [16]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import re

import os
from dotenv import load_dotenv


Importamos el API Key de las variables de entorno

In [17]:
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

In [3]:
# pip install langchain_community

In [4]:
# pip install pypdf

In [5]:
# pip install sentence-transformers

In [6]:
# !pip install chromadb

In [7]:
# pip install python-dotenv

In [8]:
# pip install google-generativeai


# Leer el documentos

In [19]:
def cargar_documentos(ruta_archivo):
    """Carga documentos en PDF, TXT o DOCX y los convierte en texto."""
    if ruta_archivo.endswith(".pdf"):
        loader = PyPDFLoader(ruta_archivo)
    elif ruta_archivo.endswith(".txt"):
        loader = TextLoader(ruta_archivo)
    elif ruta_archivo.endswith(".docx"):
        loader = Docx2txtLoader(ruta_archivo)
    else:
        raise ValueError("Formato no soportado. Usa PDF, TXT o DOCX.")
    
    documentos = loader.load()
    
    return documentos

In [20]:
# Función de limpieza
def clean_text(text):
    # Eliminar fechas en formato dd/mm/yyyy
    text = re.sub(r'\d{2}/\d{2}/\d{4}', '', text)
    
    # Eliminar metadatos innecesarios
    text = re.sub(r'USUARIO|PScript5\.dll.*?\n|Acrobat Distiller.*?\n', '', text)
    
    # Reemplazar caracteres especiales y símbolos unicode, excluyendo caracteres españoles
    text = re.sub(r'[\uf06e\uf0a7]|[^\x00-\x7F\xC0-\xFF]+', '-', text)
    
    # Eliminar múltiples espacios en blanco
    text = re.sub(r'\s+', ' ', text)
    
    # Eliminar saltos de línea innecesarios, pero mantener párrafos
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Eliminar espacios al inicio y final de cada línea
    text = '\n'.join(line.strip() for line in text.split('\n'))
    
    # Eliminar espacios en blanco al inicio y final del texto
    text = text.strip()
    
    # Eliminar caracteres especiales y símbolos repetidos
    text = re.sub(r'[-]{2,}', '-', text)
    text = re.sub(r'[.]{2,}', '.', text)
    
    return text

# page = cargar_documentos("data/1-01-Curso_PLN.pdf")

# # Limpiar cada página
# cleaned_pages = [clean_text(pag.page_content) for pag in page]

# # Unir todas las páginas en un solo texto limpio
# final_text = "\n\n".join(cleaned_pages)

# # Mostrar el resultado limpio
# print(final_text)


# Split document

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Paso 1: Convertir el texto limpio en un objeto Document
# Asegúrate de que final_text es un string que contiene el contenido limpio
# document = Document(page_content=final_text)  # Debe ser un objeto Document

# Paso 2: Función para dividir el texto en fragmentos
def split_text(document):
    """Divide el texto en fragmentos más pequeños para procesamiento."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,      # Número de caracteres por fragmento
        chunk_overlap=50,    # Traslape entre fragmentos
        length_function=len, # Función de longitud
        separators=["\n\n", "\n", " "]  # Separadores
    )

    # Aplicar el splitter al documento (documento debe ser una lista)
    textos_fragmentados = text_splitter.split_documents([document])  # Pasamos una lista de documentos

    return textos_fragmentados

# Paso 3: Ejecutar la función y obtener los fragmentos
# chunks = split_text(document)

# Mostrar un fragmento de ejemplo
# for i, chunk in enumerate(chunks[:]):  # Mostramos solo los 3 primeros
#     print(f"\nFragmento {i+1}:\n{chunk.page_content}\n{'-'*50}")

# Crear embeddings

Se hará uso de un modelo de Hugging Face all-MiniLM-L6-v2

In [22]:
# Usar modelo de Hugging Face
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Almacenar  embeddings en ChromaDB

In [23]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Lista de valores unicos para documentos
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    #Crea una database de chroma
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [14]:
# vectorstore = create_vectorstore(chunks=chunks, 
#                                  embedding_function=embeddings, 
#                                  vectorstore_path="./vectorstore")

# Definimos el LLM

In [29]:
import google.generativeai as genai

def generate_response(final_prompt):
    # Inicializa el modelo Gemini
    model = genai.GenerativeModel('gemini-2.0-flash-001')

    # Genera una respuesta
    response = model.generate_content(final_prompt)

    return response.text


# Consulta de datos relevantes

In [15]:
#Cargamos el vectorstore
# database = Chroma(persist_directory="vectorstore",embedding_function=embeddings)

In [44]:
from langchain_core.prompts import ChatPromptTemplate

def generate_materials_by_section(vectorstore):
    """
    Genera materiales educativos dividiendo el proceso en diferentes prompts específicos.
    """
    # Configurar el retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    
    prompts = {
        "title": "¿Cúal es el mejor título para el material didáctico?",
        "topics": "¿Cuáles son los temas principales que se cubren en el curso?",
        "objectives": "¿Cuáles son los objetivos de aprendizaje de este curso?",
        "resources": "¿Cuáles son las lecturas o recursos recomendados para este curso?",
        "discussion_questions": "¿Cuáles son algunas preguntas para discusión en este curso?",
        "practice_problems": "¿Qué ejercicios o problemas de práctica se incluyen en el programa del curso?"
    }
    
    retrieved_data = {key: retriever.invoke(query) for key, query in prompts.items()}
    
    prompt_templates = {
        "notas_clase": """
        Eres un asistente para la generación de materiales educativos basados en un programa de curso.
        Utiliza la siguiente información recuperada para crear notas de clase estructuradas:

        ---
        **Título del curso:** {title}
        **Temas principales:** {topics}
        
        **Notas detalladas de clase:**
        - Organiza los temas principales en secciones claras.
        - Incluye explicaciones concisas, ejemplos prácticos y aplicaciones relevantes.
        - Evita introducciones genéricas o frases como "Dado que la información es limitada".
        """,
        
        "problemas_practica": """
        Eres un asistente para la generación de materiales educativos basados en un programa de curso.
        Utiliza la siguiente información recuperada para generar problemas de práctica con soluciones:
        
        ---
        **Título del curso:** {title}
        **Ejercicios y problemas de práctica:** {practice_problems}
        
        **Problemas de práctica con soluciones:**
        - Diseña problemas que refuercen los conceptos clave del curso.
        - Incluye soluciones detalladas y explicaciones paso a paso.
        """,
        
        "preguntas_discusion": """
        Eres un asistente para la generación de materiales educativos basados en un programa de curso.
        Utiliza la siguiente información recuperada para generar preguntas de discusión:
        
        ---
        **Título del curso:** {title}
        **Preguntas para discusión:** {discussion_questions}
        
        **Preguntas para discusión:**
        - Propón preguntas que fomenten el análisis crítico y la reflexión sobre los temas del curso.
        - Asegúrate de que las preguntas estén relacionadas directamente con los objetivos de aprendizaje.
        """,
        
        "objetivos_aprendizaje": """
        Eres un asistente para la generación de materiales educativos basados en un programa de curso.
        Utiliza la siguiente información recuperada para definir objetivos de aprendizaje:
        
        ---
        **Título del curso:** {title}
        **Objetivos de aprendizaje:** {objectives}
        
        **Objetivos de aprendizaje específicos para cada tema:**
        - Define objetivos claros y medibles para cada tema principal.
        - Relaciona los objetivos con las habilidades y conocimientos que los estudiantes deben adquirir.
        """,
        
        "lecturas_sugeridas": """
        Eres un asistente para la generación de materiales educativos basados en un programa de curso.
        Utiliza la siguiente información recuperada para recomendar lecturas y recursos:
        
        ---
        **Título del curso:** {title}
        **Lecturas y recursos recomendados:** {resources}
        
        **Lecturas y recursos sugeridos:**
        - Proporciona una lista de lecturas, artículos, libros y recursos en línea que complementen los temas del curso.
        - Incluye enlaces o referencias directas cuando sea posible.
        """
    }
    
    # Generar los prompts finales con la información obtenida
    final_prompts = {
        key: ChatPromptTemplate.from_template(template).format(**retrieved_data)
        for key, template in prompt_templates.items()
    }
    
    # Generar las respuestas utilizando el modelo Gemini
    generated_materials = {
        key: generate_response(prompt)
        for key, prompt in final_prompts.items()
    }
    
    return generated_materials

# Exportar a pdf

In [55]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
import re

# Estilo para los bloques de código con fondo azul y texto blanco
code_block_style = ParagraphStyle(
    name="CodeBlockStyle",
    fontName="Courier",
    fontSize=9,
    leading=11,
    spaceBefore=4,
    spaceAfter=4,
    backColor="#E0F7FA",  # Fondo azul claro
    textColor="#01579B"    # Texto color azul oscuro
)

# Estilo para texto normal, con fondo blanco y texto negro
normal_text_style = ParagraphStyle(
    name="NormalTextStyle",
    fontName="Helvetica",
    fontSize=10,
    leading=12,
    spaceBefore=6,
    spaceAfter=6,
    textColor="#212121"  # Texto en gris oscuro
)

# Estilo para los títulos de las secciones con color de fondo y texto personalizado
section_title_style = ParagraphStyle(
    name="SectionTitleStyle",
    fontName="Helvetica-Bold",
    fontSize=14,
    leading=16,
    spaceBefore=10,
    spaceAfter=10,
    textColor="#FFFFFF",  # Texto blanco
    backColor="#00796B"   # Fondo verde oscuro
)

def convert_to_html_bold(text):
    """Convierte texto en negrita usando etiquetas HTML <b>."""
    return re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', text)

def extract_code_and_text(input_text):
    """Separa el texto en bloques de código y texto normal detectando `````."""
    parts = re.split(r"```", input_text)
    formatted_parts = []
    
    for idx, part in enumerate(parts):
        if idx % 2 == 0:
            formatted_parts.append(("text", convert_to_html_bold(part.strip())))
        else:
            # Código, reemplazando saltos de línea y espacios
            formatted_code = part.strip().replace("\n", "<br/>")
            formatted_code = formatted_code.replace(" ", "&nbsp;")
            formatted_parts.append(("code", formatted_code))
    
    return formatted_parts

def generate_material_pdf(materials_dict, pdf_filename):
    """Genera un archivo PDF con los materiales procesados y formateados."""
    pdf_filename = pdf_filename + ".pdf"
    document = SimpleDocTemplate(pdf_filename, pagesize=letter)
    styles = getSampleStyleSheet()
    content_elements = []

    # Agregar un título general al PDF
    header = Paragraph("<b>Material Didáctico Generado</b>", styles["Title"])
    content_elements.append(header)
    content_elements.append(Spacer(1, 0.3 * inch))  # Espaciado para el título

    # Procesar cada sección de los materiales
    for section_name, section_text in materials_dict.items():
        # Añadir el título de la sección con un fondo verde oscuro y texto blanco
        section_header = Paragraph(f"<b>{section_name.replace('_', ' ').title()}</b>", section_title_style)
        content_elements.append(section_header)
        content_elements.append(Spacer(1, 0.2 * inch))  # Espaciado entre título de sección

        # Procesar las secciones de texto y código
        sections = extract_code_and_text(section_text)
        for type_of_section, content in sections:
            if type_of_section == "text":
                paragraphs = content.split("\n\n")  # Separar en párrafos
                for para in paragraphs:
                    content_elements.append(Paragraph(para, normal_text_style))  # Texto normal
                    content_elements.append(Spacer(1, 0.1 * inch))  # Espaciado entre párrafos
            elif type_of_section == "code":
                content_elements.append(Paragraph(f'<font face="Courier">{content}</font>', code_block_style))  # Bloques de código
                content_elements.append(Spacer(1, 0.25 * inch))  # Espaciado después del bloque de código

    # Crear el PDF
    document.build(content_elements)
    print(f"✅ PDF generado exitosamente: {pdf_filename}")

# Pipeline

In [56]:
from tkinter import Tk, filedialog
def main():
    # Sección de input
    try:
        # Abrir diálogo para seleccionar archivo
        root = Tk()
        root.attributes('-topmost', True)
        file_path = filedialog.askopenfilename(
            parent=root,
            title="Selecciona un archivo (PDF, TXT o DOCX)",
            filetypes=(('PDF files', '*.pdf'), ('Word documents', '*.docx'), ('Text files', '*.txt'))
        )
        root.destroy()
        if not file_path:
            raise FileNotFoundError("No se seleccionó ningún archivo.")
        
        # Cargar documentos
        documents = cargar_documentos(file_path)

        # Limpiar cada página
        cleaned_pages = [clean_text(pag.page_content) for pag in documents]

        # Unir todas las páginas en un solo texto limpio
        final_text = "\n\n".join(cleaned_pages)

        # Paso 1: Convertir el texto limpio en un objeto Document
        document = Document(page_content=final_text)  # Debe ser un objeto Document

        # Paso 2: Función para dividir el texto en fragmentos
        chunks = split_text(document)

        # Paso 3: embeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

        # Paso 4: Crear vectorstore
        vectorstore = create_vectorstore(chunks=chunks, 
                                         embedding_function=embeddings, 
                                         vectorstore_path="./vectorstore")
        
        # Paso 5: Crear el prompt
        response = generate_materials_by_section(vectorstore)

        generate_material_pdf(response, input("Ingresa el nombre del archivo a generar: "))

    except FileNotFoundError as e:
        print(f"File error: {str(e)}")
    except ValueError as e:
        print(f"Input error: {str(e)}")
    except Exception as e:
        print(f"Error inesperado: {e}")

In [57]:
if __name__ == "__main__":
    main()

✅ PDF generado exitosamente: material_didactico.pdf
